# Download Newsletters from The Talon Conspiracy
 

In [ ]:
# Add path
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
#  regular imports
from utils.selenium_resource import SeleniumResource
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [39]:
ttc_url = "https://thetalonconspiracy.com/"

client = SeleniumResource()
client.setup_for_execution()

In [40]:
try:
    href_list = []
    # Navigate to the webpage
    client.driver.get(ttc_url)

    # Wait for the sidebar to be present
    sidebar = WebDriverWait(client.driver, 10).until(
        EC.presence_of_element_located((By.ID, "sidebarleft"))
    )

    # Find all anchor tags within list items in the sidebar
    # Using XPath to find all <a> tags that are descendants of <li> elements
    links = sidebar.find_elements(
        By.XPATH,
        # This finds all <a> tags with href attributes within <li> elements
        ".//li//a[@href]"
    )

    # Extract href attributes
    href_list = [link.get_attribute('href') for link in links]

    # Print results (optional)
    print(f"Found {len(href_list)} links:")
    for idx, href in enumerate(href_list, 1):
        print(f"{idx}. {href}")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    client.teardown_after_execution()

Found 556 links:
1. https://thetalonconspiracy.com/tag/the-mysterious-joseph/
2. https://thetalonconspiracy.com/tag/1940s/
3. https://thetalonconspiracy.com/tag/1970s/
4. https://thetalonconspiracy.com/tag/1980s/
5. https://thetalonconspiracy.com/tag/1990s/
6. https://thetalonconspiracy.com/tag/2000s/
7. https://thetalonconspiracy.com/tag/2001-direct-action-report/
8. https://thetalonconspiracy.com/tag/2010s/
9. https://thetalonconspiracy.com/tag/a320-campaign/
10. https://thetalonconspiracy.com/tag/aalas-convention/
11. https://thetalonconspiracy.com/tag/abortion-argument/
12. https://thetalonconspiracy.com/tag/acteal-murders/
13. https://thetalonconspiracy.com/tag/action-animal-rescue-team/
14. https://thetalonconspiracy.com/tag/adam-troy-peace/
15. https://thetalonconspiracy.com/tag/ake-soderlund/
16. https://thetalonconspiracy.com/tag/alan-cooper/
17. https://thetalonconspiracy.com/tag/alan-hoffman-snitch/
18. https://thetalonconspiracy.com/tag/alan-summersgill/
19. https://thetalo